# Technical Params

In [ ]:
!pip install thop
!pip install fvcore

In [ ]:
import torch
import torch.nn as nn
import timm

from ultralytics import YOLO
from thop import profile

In [ ]:
model_path = "/home/jovyan/data/lightning/LiviaMurankova/new/namnozene_meteory/najlepsie_modely/najlepsie_modely/yolov12_vacsie_obrazky.pt"
model = YOLO(model_path)

#print(model)
#print(model.model)
print(model.model.__class__)


<class 'ultralytics.nn.tasks.DetectionModel'>


In [ ]:
# yolov12

model = torch.load("/home/jovyan/data/lightning/LiviaMurankova/new/namnozene_meteory/najlepsie_modely/najlepsie_modely/yolov12_vacsie_obrazky.pt", map_location=torch.device("cpu"))
print(model.keys()) 

dict_keys(['date', 'version', 'license', 'docs', 'epoch', 'best_fitness', 'model', 'ema', 'updates', 'optimizer', 'train_args', 'train_metrics', 'train_results'])


In [54]:
print(model['train_args'])

{'task': 'detect', 'mode': 'train', 'model': 'runs/detect/roboflow_train/weights/best.cache.pt', 'data': '/train/cache/dataset/data.yaml', 'epochs': 300, 'time': None, 'patience': 10, 'batch': 4, 'imgsz': 1408, 'save': True, 'save_period': -1, 'cache': None, 'device': 'cuda:0', 'workers': 4, 'project': None, 'name': 'roboflow_train', 'exist_ok': False, 'pretrained': True, 'optimizer': 'auto', 'verbose': True, 'seed': 0, 'deterministic': True, 'single_cls': False, 'rect': False, 'cos_lr': False, 'close_mosaic': 300, 'resume': 'runs/detect/roboflow_train/weights/best.cache.pt', 'amp': True, 'fraction': 1.0, 'profile': False, 'freeze': None, 'multi_scale': False, 'overlap_mask': False, 'mask_ratio': 4, 'dropout': 0.0, 'val': True, 'split': 'val', 'save_json': False, 'save_hybrid': False, 'conf': None, 'iou': 0.7, 'max_det': 300, 'half': False, 'dnn': False, 'plots': True, 'source': None, 'vid_stride': 1, 'stream_buffer': False, 'visualize': False, 'augment': False, 'agnostic_nms': False, 

In [77]:
print(model['docs'])

https://docs.ultralytics.com


In [ ]:
model = YOLO("/home/jovyan/data/lightning/LiviaMurankova/new/namnozene_meteory/najlepsie_modely/najlepsie_modely/yolov12_vacsie_obrazky.pt")

model.eval()

# Dummy input of size 1408 x 1408
dummy_input = torch.randn(1, 3, 1408, 1408)

flops, params = profile(model.model, inputs=(dummy_input,), verbose=False)

print(f"FLOPs: {flops / 1e9:.2f} GFLOPs")
print(f"Parametre: {params / 1e6:.2f} miliónov")

FLOPs: 52.08 GFLOPs
Parametre: 9.26 miliónov


In [ ]:
model = torch.load("/home/jovyan/data/lightning/LiviaMurankova/new/namnozene_meteory/najlepsie_modely/najlepsie_modely/yolov11_vacsie_obrazky.pt", map_location=torch.device("cpu"))

def count_conv_layers(model):
    conv_count = sum(1 for layer in model.modules() if isinstance(layer, nn.Conv2d))
    return conv_count

conv_layers_count = count_conv_layers(model['model'])
print(conv_layers_count)

88


In [ ]:
model = timm.create_model("convnext_small", pretrained=True, num_classes=1)

model_path = "/home/jovyan/data/lightning/LiviaMurankova/new/namnozene_meteory/najlepsie_modely/najlepsie_modely/datasetv2_best_convnext_model_v5.pth"
state_dict = torch.load(model_path, map_location=torch.device("cpu"))

# get rid of bias
state_dict.pop("classifier.bias", None)

# load weights
model.load_state_dict(state_dict, strict=False)

def count_conv_layers(model):
    return sum(1 for layer in model.modules() if isinstance(layer, torch.nn.Conv2d))

conv_layers_count = count_conv_layers(model)
print(f"Počet konvolučných vrstiev: {conv_layers_count}")


Počet konvolučných vrstiev: 40


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# EfficientNet-B4
model_effnet = timm.create_model("efficientnet_b4", pretrained=True, num_classes=1)
print(f"EfficientNet-B4: {count_parameters(model_effnet) / 1e6:.2f}M parametrov")

# ConvNeXt-Small
model_convnext = timm.create_model("convnext_small", pretrained=True, num_classes=1)
print(f"ConvNeXt-Small: {count_parameters(model_convnext) / 1e6:.2f}M parametrov")

model.safetensors:   0%|          | 0.00/77.9M [00:00<?, ?B/s]

EfficientNet-B4: 17.55M parametrov
ConvNeXt-Small: 49.46M parametrov


In [ ]:


efficientnet_b4 = timm.create_model("efficientnet_b4", pretrained=False)
convnext_small = timm.create_model("convnext_small", pretrained=False)

# Input tensor of size (1, 3, 720, 720)
input_tensor = torch.randn(1, 3, 720, 720)

# Count FLOPs and params
flops_efficientnet, params_efficientnet = profile(efficientnet_b4, inputs=(input_tensor,))
flops_convnext, params_convnext = profile(convnext_small, inputs=(input_tensor,))

print(f"EfficientNet-B4: {flops_efficientnet / 1e9:.2f} GFLOPs, {params_efficientnet / 1e6:.2f}M parametrov")
print(f"ConvNeXt-Small: {flops_convnext / 1e9:.2f} GFLOPs, {params_convnext / 1e6:.2f}M parametrov")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


[W409 19:38:26.495242584 NNPACK.cpp:62] Could not initialize NNPACK! Reason: Unsupported hardware.


[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
EfficientNet-B4: 15.75 GFLOPs, 19.22M parametrov
ConvNeXt-Small: 89.37 GFLOPs, 50.18M parametrov


In [ ]:
weights_path = "/home/jovyan/data/lightning/LiviaMurankova/new/namnozene_meteory/najlepsie_modely/najlepsie_modely/yolov12_vacsie_obrazky.pt"
checkpoint = torch.load(weights_path, map_location='cpu')

# model weights
state_dict = checkpoint['model'].state_dict()

total_params = sum(p.numel() for p in state_dict.values())
print(f"Počet parametrov: {total_params:,} ({total_params / 1e6:.2f} milióna)")


Počet parametrov: 9,299,684 (9.30 milióna)


In [ ]:
# SE Block
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction),
            nn.ReLU(),
            nn.Linear(in_channels // reduction, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y

# CNN model w SE blocks
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.se1 = SEBlock(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.se2 = SEBlock(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.se3 = SEBlock(128)
        self.dropout = nn.Dropout(0.3)

        # count input size - conv + pool
        sample_input = torch.randn(1, 3, 320, 480)
        with torch.no_grad():
            feature_size = self._get_feature_map_size(sample_input)

        self.fc1 = nn.Linear(feature_size, 128)
        self.fc2 = nn.Linear(128, 1)

    def _get_feature_map_size(self, x):
        x = torch.relu(self.conv1(x))
        x = self.se1(x)
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = self.se2(x)
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x))
        x = self.se3(x)
        x = torch.max_pool2d(x, 2)
        return x.view(1, -1).size(1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.se1(x)
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = self.se2(x)
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv3(x))
        x = self.se3(x)
        x = torch.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = torch.relu(self.fc1(x))
        return torch.sigmoid(self.fc2(x))

model = CNNModel()
model.eval()

dummy_input = torch.randn(1, 3, 480, 480)

flops, params = profile(model, inputs=(dummy_input,), verbose=False)

print(f"FLOPs: {flops / 1e9:.2f} GFLOPs")
print(f"Parametre: {params / 1e6:.2f} miliónov")

FLOPs: 1.60 GFLOPs
Parametre: 39.42 miliónov
